In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/priject1/lab7/wheat-seeds.csv"
df = pd.read_csv(path)
print('\nNumber of rows and columns in the data set: ',df.shape)


Number of rows and columns in the data set:  (210, 8)


In [ ]:
df.head()

,f1,f2,f3,f4,f5,f6,f7,f8
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [ ]:
X = df.drop("f8", axis=1)
y = df["f8"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class Neuralnet:
    def __init__(self, neurons):
        self.layers = len(neurons)
        self.rate = .01
        self.inputs = []
        self.outputs = []
        self.errors = []
        self.weights = []
        self.biases = []

        for layer in range(self.layers):
            self.inputs.append(np.empty(neurons[layer]))
            self.outputs.append(np.empty(neurons[layer]))
            self.errors.append(np.empty(neurons[layer]))

        for layer in range(self.layers - 1):
            self.weights.append(np.random.normal(
                scale=1.0/np.sqrt(neurons[layer]),
                size=[neurons[layer], neurons[layer + 1]]
            ))
            self.biases.append(np.random.normal(
                scale=1.0/np.sqrt(neurons[layer]),
                size=neurons[layer + 1]
            ))

    def feedforward(self, inputs):
        self.inputs[0] = inputs
        for layer in range(self.layers - 1):
            self.outputs[layer] = np.tanh(self.inputs[layer])
            self.inputs[layer + 1] = np.dot(self.weights[layer].T, self.outputs[layer]) + self.biases[layer]
        self.outputs[-1] = np.tanh(self.inputs[-1])

    def backpropagate(self, targets):
        self.errors[-1] = self.outputs[-1] - targets
        for layer in reversed(range(self.layers - 1)):
            gradient = 1 - self.outputs[layer] * self.outputs[layer]
            self.errors[layer] = gradient * np.dot(self.weights[layer], self.errors[layer + 1])
        for layer in range(self.layers - 1):
            self.weights[layer] -= self.rate * np.outer(self.outputs[layer], self.errors[layer + 1])
            self.biases[layer] -= self.rate * self.errors[layer + 1]



In [ ]:

net = Neuralnet([7 * 1, 200, 3])

y_label = y_train.to_numpy()
data = X_train

for epoch in range(20):
    error = 0
    for sample in range(len(data)):
        net.feedforward(data.iloc[sample])
        targets = np.full(3, -1, dtype= int)
        targets[y_label[sample]-1] = 1
        net.backpropagate(targets)
        error += np.sum(net.errors[-1] * net.errors[-1])

test = X_train
predictions = []
for i in range(len(test)):
    net.feedforward(test.iloc[i])
    predictions.append(np.argmax(net.outputs[-1]))

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, predictions)*100

83.92857142857143


In [12]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(random_state=1, max_iter=1000).fit(X_train, y_train)
clf.predict_proba(X_test)
clf.predict(X_test)

clf.score(X_test, y_test)

0.9047619047619048